In [11]:
import cv2
import numpy as np
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers import Lambda, Cropping2D

In [12]:
lines = []
with open("C:\\Users\\deadl\\Desktop\\udacity\\driving_log.csv") as csvFile:
    fileData = csv.reader(csvFile)
    for line in fileData:
        lines.append(line)
    

In [13]:
lines

[['C:\\Users\\deadl\\Desktop\\udacity\\IMG\\center_2020_02_25_23_14_16_824.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\left_2020_02_25_23_14_16_824.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\right_2020_02_25_23_14_16_824.jpg',
  '0',
  '0',
  '0',
  '3.872785E-06'],
 ['C:\\Users\\deadl\\Desktop\\udacity\\IMG\\center_2020_02_25_23_14_16_886.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\left_2020_02_25_23_14_16_886.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\right_2020_02_25_23_14_16_886.jpg',
  '0',
  '0',
  '0',
  '1.279975E-05'],
 ['C:\\Users\\deadl\\Desktop\\udacity\\IMG\\center_2020_02_25_23_14_16_987.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\left_2020_02_25_23_14_16_987.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\right_2020_02_25_23_14_16_987.jpg',
  '0',
  '0',
  '0',
  '6.043104E-06'],
 ['C:\\Users\\deadl\\Desktop\\udacity\\IMG\\center_2020_02_25_23_14_17_062.jpg',
  'C:\\Users\\deadl\\Desktop\\udacity\\IMG\\left_2020_02_25_23_14_17_062.jpg',


In [14]:
images = []
steer = []
for line in lines:
    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_center = float(line[3])
    steering_left = steering_center + correction
    steering_right = steering_center - correction
    
    images.append(cv2.imread(line[0]))
    steer.append(steering_center)
    images.append(cv2.flip(cv2.imread(line[0]),1))
    steer.append(steering_center*-1)
    
    images.append(cv2.imread(line[1]))
    steer.append(steering_left)
    images.append(cv2.flip(cv2.imread(line[1]),1))
    steer.append(steering_left*-1)
    
    images.append(cv2.imread(line[2]))
    steer.append(steering_right)
    images.append(cv2.flip(cv2.imread(line[2]),1))
    steer.append(steering_right*-1)


In [16]:
X = np.array(images)
Y = np.array(steer)

In [17]:
model = Sequential()

# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

# trim image to only see section with road
model.add(Cropping2D(cropping=((70,25),(0,0))))           


model.add(Convolution2D(6,5,5))
model.add(Activation('relu'))


model.add(Convolution2D(6,5,5))
model.add(Activation('relu'))

#flatten image from 2D to side by side
model.add(Flatten())


model.add(Dense(20))
model.add(Activation('relu'))



model.add(Dense(84))
model.add(Activation('relu'))

model.add(Dense(1)) #here the final layer will contain one value as this is a regression problem and not classification


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5))`
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5))`
  


In [18]:
model.compile(loss= "mse",optimizer="adam")

In [20]:
model.fit(X,Y,validation_split=.3,shuffle=True,epochs=10)

Train on 3981 samples, validate on 1707 samples
Epoch 1/10
3981/3981 [==============================] - 70s 18ms/step - loss: 0.0668 - val_loss: 0.2155
Epoch 2/10
3981/3981 [==============================] - 70s 18ms/step - loss: 0.0454 - val_loss: 0.2478
Epoch 3/10
3981/3981 [==============================] - 70s 18ms/step - loss: 0.0201 - val_loss: 0.2778
Epoch 4/10
3981/3981 [==============================] - 71s 18ms/step - loss: 0.0075 - val_loss: 0.2825
Epoch 5/10
3981/3981 [==============================] - 70s 18ms/step - loss: 0.0031 - val_loss: 0.2768
Epoch 6/10
3981/3981 [==============================] - 73s 18ms/step - loss: 0.0017 - val_loss: 0.2917
Epoch 7/10
3981/3981 [==============================] - 72s 18ms/step - loss: 0.0013 - val_loss: 0.2733
Epoch 8/10
3981/3981 [==============================] - 73s 18ms/step - loss: 0.0017 - val_loss: 0.2798
Epoch 9/10
3981/3981 [==============================] - 75s 19ms/step - loss: 0.0021 - val_loss: 0.2741
Epoch 10/10
3981

In [21]:
model.save("model.h5")